In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv( "/content/drive/MyDrive/Training.csv" )
df

FileNotFoundError: [Errno 2] File /content/drive/MyDrive/Training.csv does not exist: '/content/drive/MyDrive/Training.csv'

# Conversion a Dolar

In [ ]:
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df['Total_Amount']*0.0096
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df['Total_Amount']*1.17
df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = df['Total_Amount']*0.70
df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df['Total_Amount']*1.29
df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USData'

In [ ]:
short_df = df[['Region', 'Total_Amount', 'Delivery_Year', 'Delivery_Quarter','Stage' ]].rename(columns={'Stage': 'Decision'})
short_df = short_df[ (short_df['Decision'] == 'Closed Won') | (short_df['Decision'] == 'Closed Lost') ]

# Probanding

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [ ]:
X = short_df.drop( 'Decision', axis=1 )
y = short_df['Decision']
X = X.drop(columns='Total_Amount')
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_train

In [ ]:
ohe = OneHotEncoder()
ohe.fit(X_train)
X_train_enc = ohe.transform(X_train)
X_test_enc = ohe.transform(X_test)

In [ ]:
rfc = RandomForestClassifier(n_estimators=1000)
#n_estimators se refiere a la cantidad de arboles de decisiones que usa el rfc

In [ ]:
rfc.fit(X_train_enc, y_train)

In [ ]:
rfc_pred = rfc.predict(X_test_enc)

In [ ]:
print(classification_report(y_test, rfc_pred))
print(confusion_matrix(y_test, rfc_pred))